In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

file_path = "baby weight regression.xlsx"
use_col = "B:P"
target_col = "Birthweight"
sheet_name = "Birthweight_reduced_kg_R"


# Read Excel Data
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=use_col)
# Remove the row with null target column
df[target_col] = df[target_col].replace(r'^\s*$', np.nan, regex=True)
print("Data Overview：")
print(df.head())

Data Overview：
   Length  Headcirc  Gestation  smoker  mage  mnocig  mheight  mppwt  fage  \
0      56        34         44       0    20       0      162     57    23   
1      53        36         40       0    19       0      171     62    19   
2      58        39         41       0    35       0      172     58    31   
3      53        38         44       0    20       0      174     68    26   
4      54        37         42       0    24       0      175     66    30   

   fedyrs  fnocig  fheight  lowbwt  mage35  Birthweight  
0      10      35      179       0       0         4.55  
1      12       0      183       0       0         4.32  
2      16      25      185       0       1         4.10  
3      14      25      189       0       0         4.07  
4      12       0      184       0       0         3.94  


In [2]:
# Build Correlation Matrix
def GetCorrelationMatrix(df):
    print("\nCorrelation Matrix：")
    correlation_matrix = df.corr()
    print(correlation_matrix)
    return correlation_matrix

# Build Regression Model
def GetRegressionModel(X, y, df):
    X = df[X]
    y = df[y]
    
    # Add a constant term (intercept)
    X = sm.add_constant(X)
    
    # Create model and fit
    model = sm.OLS(y, X).fit()
    
    # Print Regression Result
    print("\n Regression Result：")
    print(model.summary())
    
    # use scikit-learn for linear regression
    simple_model = LinearRegression()
    simple_model.fit(X, y)
    print(f"\nR² score（scikit-learn）：{simple_model.score(X, y):.4f}")
    return model

# Print Prediction and Error
def PrintResiduals(model, y):
    y = df[y]
    print("\nRESIDUAL OUTPUT：")
    residuals = model.resid  # Get Residuals
    fitted_values = model.fittedvalues  # Get Predictoin
    error = abs((fitted_values - y) / y)
    
    # Create Result DataFrame with predicted value and error
    residual_output = pd.DataFrame({
        'Observation': y,
        'Predicted '+target_col: fitted_values,
        #'Residuals': residuals,
        'Error': error
    })
    
    print(residual_output)


In [3]:
# Print the Correlation Matrix and Regression results for original dataset
independent_vars = [c for c in df.columns if c != target_col]
orig_corr_mat = GetCorrelationMatrix(df)
orig_model = GetRegressionModel(independent_vars, target_col, df)
PrintResiduals(orig_model, target_col)


Correlation Matrix：
               Length  Headcirc  Gestation        smoker      mage    mnocig  \
Length       1.000000  0.563172   0.705111 -1.534062e-01  0.075268 -0.039843   
Headcirc     0.563172  1.000000   0.404635 -1.828719e-01  0.145842 -0.132988   
Gestation    0.705111  0.404635   1.000000 -9.474608e-02  0.010778  0.043195   
smoker      -0.153406 -0.182872  -0.094746  1.000000e+00  0.212479  0.727218   
mage         0.075268  0.145842   0.010778  2.124788e-01  1.000000  0.340294   
mnocig      -0.039843 -0.132988   0.043195  7.272181e-01  0.340294  1.000000   
mheight      0.484992  0.337047   0.210503  3.532676e-04  0.059956  0.126439   
mppwt        0.398197  0.302854   0.255082  1.006136e-15  0.274168  0.148945   
fage         0.137184  0.301151   0.142175  1.975014e-01  0.806584  0.248425   
fedyrs       0.079485  0.123892   0.130987 -1.489058e-02  0.441683  0.198526   
fnocig       0.008800 -0.046837  -0.113831  4.176330e-01  0.090927  0.257307   
fheight      0.2083

In [4]:
# Check p value and correlation value to see if there is any column that need to be drop
# p value should < 0.05, correlation value should <= -0.8 or >= 0.8

large_p_value = list(orig_model.pvalues[orig_model.pvalues>0.05].index)
small_corr = list(orig_corr_mat[target_col][orig_corr_mat[target_col]<0.8].index)
drop_col = large_p_value or small_corr
drop_col = [var for var in drop_col if var != 'const']
print("p value > 0.05:")
print(orig_model.pvalues[orig_model.pvalues>0.05].drop(['const']))
print("\n-0.8 < correlation < 0.8:")
print(orig_corr_mat[target_col][abs(orig_corr_mat[target_col])<0.8])
print("\nColumn might be removed:")
print(drop_col)

p value > 0.05:
Length     0.220102
smoker     0.206392
mage       0.629278
mnocig     0.785029
mheight    0.849124
mppwt      0.431920
fage       0.676416
fedyrs     0.850922
fnocig     0.605816
fheight    0.135564
lowbwt     0.890390
mage35     0.155429
dtype: float64

-0.8 < correlation < 0.8:
Length       0.726833
Headcirc     0.684616
Gestation    0.708303
smoker      -0.314234
mage         0.000173
mnocig      -0.152335
mheight      0.363055
mppwt        0.400886
fage         0.175710
fedyrs       0.071045
fnocig      -0.093136
fheight      0.031022
lowbwt      -0.651964
mage35      -0.108947
Name: Birthweight, dtype: float64

Column might be removed:
['Length', 'smoker', 'mage', 'mnocig', 'mheight', 'mppwt', 'fage', 'fedyrs', 'fnocig', 'fheight', 'lowbwt', 'mage35']


In [5]:
# if drop_col is incorrect, please set it manually

new_indep_vars = [var for var in independent_vars if var not in drop_col]
new_df = df.drop(drop_col,axis=1)
# print(new_df)
new_corr_mat = GetCorrelationMatrix(new_df)
new_model = GetRegressionModel(new_indep_vars, target_col, new_df)
PrintResiduals(new_model, target_col)


Correlation Matrix：
             Headcirc  Gestation  Birthweight
Headcirc     1.000000   0.404635     0.684616
Gestation    0.404635   1.000000     0.708303
Birthweight  0.684616   0.708303     1.000000

 Regression Result：
                            OLS Regression Results                            
Dep. Variable:            Birthweight   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.675
Method:                 Least Squares   F-statistic:                     43.63
Date:                Sat, 12 Apr 2025   Prob (F-statistic):           1.12e-10
Time:                        17:01:34   Log-Likelihood:                -13.236
No. Observations:                  42   AIC:                             32.47
Df Residuals:                      39   BIC:                             37.68
Df Model:                           2                                         
Covariance Type:            nonrobust                          